In [18]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split



In [16]:
cols = ['Age', 
        'Gender',
        'Country',
        'self_employed',
        'work_interfere', 
        'no_employees',
        'remote_work',
        'tech_company',
        'benefits',
        'care_options',
        'wellness_program',
        'seek_help',
        'leave',
        'coworkers',
        'mental_vs_physical', 
        'mental_health_consequence']

unprocessed_data = pd.read_csv('survey.csv', usecols=cols)

In [17]:
unprocessed_data.head()

,Age,Gender,Country,self_employed,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,leave,mental_health_consequence,coworkers,mental_vs_physical
0,37,Female,United States,NaN,Often,6-25,No,Yes,Yes,Not sure,No,Yes,Somewhat easy,No,Some of them,Yes
1,44,M,United States,NaN,Rarely,More than 1000,No,No,Don't know,No,Don't know,Don't know,Don't know,Maybe,No,Don't know
2,32,Male,Canada,NaN,Rarely,6-25,No,Yes,No,No,No,No,Somewhat difficult,No,Yes,No
3,31,Male,United Kingdom,NaN,Often,26-100,No,Yes,No,Yes,No,No,Somewhat difficult,Yes,Some of them,No
4,31,Male,United States,NaN,Never,100-500,Yes,Yes,Yes,No,Don't know,Don't know,Don't know,No,Some of them,Don't know


In [19]:
len(unprocessed_data)

1259

In [35]:
unprocessed_mat = unprocessed_data.values.tolist()
age = []
male_female = []
trans = []

in_states = []

self_employed_yes_no = []
self_employed_na = []

work_interfere_often = []
work_interfere_rarely = []
work_interfere_na = []
work_interfere_sometimes = []
work_interfere_never = []

no_employees = []

for row in range(0, len(unprocessed_mat)):
    # age
    if unprocessed_mat[row][0] < 0 or unprocessed_mat[row][0] > 80:
        age.append(0)
    else:
        age.append(unprocessed_mat[row][0])
    
    # Gender 
    male_labels = ['m','male','male-ish', 'cis man', 'cis male', 'mal', 'male (cis)', 'man', 'malr', 'man', 'msle']
    female_labels = ['f', 'female', 'woman', 'cis female', 'cis-female/femme', 'femail', 'femake']
    if unprocessed_mat[row][1].lower() in male_labels:
        male_female.append(0)
        trans.append(0)
    elif unprocessed_mat[row][1].lower() in female_labels:
        male_female.append(1)
        trans.append(0)
    else:
        male_female.append(0)
        trans.append(1)
    
    # Location
    if unprocessed_mat[row][2] == 'United States':
        in_states.append(1)
    else:
        in_states.append(0)
    
    # self_employed
    if unprocessed_mat[row][3] == 'No':
        self_employed_yes_no.append(0)
        self_employed_na.append(0)
    elif unprocessed_mat[row][3] == 'Yes':
        self_employed_yes_no.append(1)
        self_employed_na.append(0)
    else:
        self_employed_yes_no.append(0)
        self_employed_na.append(1)
    
    # work interfere
    if unprocessed_mat[row][4] == 'Often':
        work_interfere_often.append(1)
        work_interfere_rarely.append(0)
        work_interfere_sometimes.append(0)
        work_interfere_never.append(0)
        work_interfere_na.append(0)
    elif unprocessed_mat[row][4] == 'Rarely':
        work_interfere_often.append(0)
        work_interfere_rarely.append(1)
        work_interfere_sometimes.append(0)
        work_interfere_never.append(0)
        work_interfere_na.append(0)
    elif unprocessed_mat[row][4] == 'Sometimes':
        work_interfere_often.append(0)
        work_interfere_rarely.append(0)
        work_interfere_sometimes.append(1)
        work_interfere_never.append(0)
        work_interfere_na.append(0)
    elif unprocessed_mat[row][4] == 'Never':
        work_interfere_often.append(0)
        work_interfere_rarely.append(0)
        work_interfere_sometimes.append(0)
        work_interfere_never.append(1)
        work_interfere_na.append(0)
    else:
        work_interfere_often.append(0)
        work_interfere_rarely.append(0)
        work_interfere_sometimes.append(0)
        work_interfere_never.append(0)
        work_interfere_na.append(1)
        
    # number of employees
    if unprocessed_mat[row][5] == 'More than 1000':
        no_employees.append(1000)
    else:
        arr = unprocessed_mat[row][5].split('-')
        no_employees.append(arr[0])
        
        